In [75]:
import pandas as pd
file_path = 'Add Path'
data = pd.read_excel(file_path)
print(data)

    ISO3 Code    Year         Type  Delay (days)             Last Update
0         DNK  2018.0   Timeliness          59.0 2023-10-07 14:23:38.662
1         DEU  2015.0   Timeliness          35.0 2022-12-13 14:23:38.662
2         BRA  2017.0   Timeliness           3.0 2022-08-02 14:23:38.662
3         DEU  2018.0  Periodicity           4.0 2023-10-11 14:23:38.662
4         DEU  2017.0  Periodicity          45.0 2024-05-03 14:23:38.662
..        ...     ...          ...           ...                     ...
495       IND  2021.0   Timeliness           6.0 2023-01-07 14:23:38.665
496       EGY  2016.0  Periodicity           3.0 2023-04-15 14:23:38.665
497       MEX  2017.0   Timeliness          16.0 2022-11-14 14:23:38.665
498       JPN  2019.0   Timeliness          32.0 2022-09-14 14:23:38.665
499       BRA  2015.0  Periodicity          56.0 2023-01-29 14:23:38.665

[500 rows x 5 columns]


In [73]:
# 1) Filter the Data: Only include rows where Type is Timeliness. Exclude any records where Delay (days) is negative or missing

#Filter the data for Type == 'Timeless' and Delay (days) >=0, excluding missing values 
filtered_data= data[(data['Type']== 'Timeliness') & (data['Delay (days)'] >=0)]

# Drop rows with missing values in 'Delay (days)'
filtered_data= filtered_data.dropna(subset=['Delay (days)'])

#Print
print(filtered_data.head())


  ISO3 Code    Year        Type  Delay (days)             Last Update
0       DNK  2018.0  Timeliness          59.0 2023-10-07 14:23:38.662
1       DEU  2015.0  Timeliness          35.0 2022-12-13 14:23:38.662
2       BRA  2017.0  Timeliness           3.0 2022-08-02 14:23:38.662
5       IND  2016.0  Timeliness          39.0 2022-11-09 14:23:38.662
7       DNK  2023.0  Timeliness          52.0 2024-07-28 14:23:38.662


In [79]:
# 2) Compute Aggregates: Calculate the average delay per year for each country.
#Identify the top 5 countries with the highest average delay across all years.


# Group by 'ISO3 Code' and 'Year' and calculate the average 'Delay (days)'
avg_delay_per_year= filtered_data.groupby(['ISO3 Code','Year'])['Delay (days)'].mean().reset_index()

# calculate the overall average delay for each country- all years
average_delay_per_country= avg_delay_per_year.groupby('ISO3 Code')['Delay (days)'].mean().reset_index()

# Sort by 'Delay (days)' in descending
top_5_countries = average_delay_per_country.sort_values(by='Delay (days)', ascending=False).head(5)

# Display the top 5 countries with the highest average delay
print(top_5_countries)
print(avg_delay_per_year)

  ISO3 Code  Delay (days)
8       JPN     39.606250
5       EGY     37.518519
7       IND     33.202381
0       ARG     32.481481
9       MEX     32.404762
   ISO3 Code    Year  Delay (days)
0        ARG  2015.0     20.000000
1        ARG  2016.0     55.000000
2        ARG  2017.0     33.000000
3        ARG  2018.0     22.500000
4        ARG  2019.0     10.000000
..       ...     ...           ...
74       MEX  2017.0     13.333333
75       MEX  2018.0     39.000000
76       MEX  2019.0     23.000000
77       MEX  2021.0     24.500000
78       MEX  2022.0     54.000000

[79 rows x 3 columns]


In [131]:
# 3)Regional Insights

#Compute the average delay by region for each year.
#Identify which region has shown the most improvement in timeliness over the years.

region_mapping = {
    'ARG': 'Latin America & Caribbean',
    'BRA': 'Latin America & Caribbean',
    'CAN': 'North America',
    'DEU': 'Europe',
    'DNK': 'Europe',
    'EGY': 'Middle East & North Africa',
    'FRA': 'Europe',
    'IND': 'South Asia',
    'JPN': 'East Asia & Pacific',
    'MEX': 'Latin America & Caribbean'
}

# Apply region mapping to the filtered data
filtered_data['Region'] = filtered_data['ISO3 Code'].map(region_mapping)

In [129]:
# Compute the average delay per region for each year
region_avg_delay = filtered_data.groupby(['Region', 'Year'])['Delay (days)'].mean().reset_index()

# Display the average delay by region for each year
print(region_avg_delay)

                        Region    Year  Delay (days)
0          East Asia & Pacific  2015.0     28.000000
1          East Asia & Pacific  2016.0     33.666667
2          East Asia & Pacific  2017.0     45.000000
3          East Asia & Pacific  2018.0     59.000000
4          East Asia & Pacific  2019.0     38.333333
5          East Asia & Pacific  2020.0     42.250000
6          East Asia & Pacific  2022.0     45.000000
7          East Asia & Pacific  2023.0     25.600000
8                       Europe  2015.0     28.727273
9                       Europe  2016.0     32.500000
10                      Europe  2017.0     23.800000
11                      Europe  2018.0     43.666667
12                      Europe  2019.0     24.000000
13                      Europe  2020.0     36.000000
14                      Europe  2021.0     41.500000
15                      Europe  2022.0     27.833333
16                      Europe  2023.0     42.375000
17   Latin America & Caribbean  2015.0     32.

In [59]:
# Calculate the difference in average delay between the first and last year for each region
region_improvement = region_avg_delay.groupby('region').agg(
    start_delay=('Delay (days)', 'first'),
    end_delay=('Delay (days)', 'last')
).reset_index()

region_improvement['improvement'] = region_improvement['start_delay'] - region_improvement['end_delay']

# the region with the most improvement
most_improved_region = region_improvement.loc[region_improvement['improvement'].idxmax()]

print(f"Region with most improvement: {most_improved_region['region']}")
print(f"Improvement: {most_improved_region['improvement']} days")


Region with most improvement: Middle East & North Africa
Improvement: 17.333333333333336 days


In [100]:
# 4)Outlier Detection: Identify any countries with delays that are statistical outliers

from scipy.stats import zscore

# Calculate Z-scores for 'Delay (days)'
filtered_data['z_score']=zscore(filtered_data['Delay (days)'])

In [126]:
# Identify countries with Z-scores, I had to use 1.8 for detecting outliers 
#A lower threshold like 1.8 has a balance between being too lenient (±3) and too strict (±1).

outliers_zscore=filtered_data[filtered_data['z_score'].abs()>1.8]


In [127]:
# Display the outliers
print(outliers_zscore[['ISO3 Code', 'Year', 'Delay (days)', 'z_score']])


    ISO3 Code    Year  Delay (days)   z_score
293       ARG  2018.0           0.0 -1.838781
390       MEX  2017.0           0.0 -1.838781


In [64]:
#Export
avg_delay_per_year.to_csv('Average_Delay_By_Country.csv', index=False)
region_avg_delay.to_csv('Regional_Averages.csv', index=False)
outliers_zscore.to_csv('Outliers.csv', index=False)

Average_Delay_By_Country.csv exported.
